In [1]:
import numpy as np
import pandas as pd
data_path="../data/ml-100k/u.data"
data_fields = ['user_id', 'item_id', 'rating', 'timestamp']
# all data file
data_df = pd.read_table(data_path, names=data_fields)

# get user number
n_users = max(data_df['user_id'].values)
# get item number
n_items = max(data_df['item_id'].values)

In [8]:
from scipy.sparse import coo_matrix
# S = dok_matrix((5, 5), dtype=np.float32)
data = np.ones((data_df.shape[0]))
row = data_df.user_id-1
col = data_df.item_id-1
UI = coo_matrix((data, (row, col)), shape=(n_users, n_items))
UIUI = UI.dot(UI.transpose()).dot(UI)

In [13]:
UIUI=UIUI.tocoo()
UI=UI.tocoo()

In [29]:
M.data.astype(np.int)

array([   77,   174,   174, ..., 16281, 12599, 19796])

In [30]:
def save_M(M, M_str):
    df=pd.DataFrame()
    df['row']=M.row
    df['col']=M.col
    df['data']=M.data.astype(np.int)
    df.to_csv(M_str,header=None,index=False,sep='\t')
    
save_M(UIUI,'hin/UIUI.txt')
save_M(UI,'hin/UI.txt')

In [1]:
from lfm_new_data import getDataLoader,LFM
import numpy as np
import pandas as pd

from utils import Interactions
import torch
SMALL=False
IMPLICT=True

def drop_df(df):
    pos_cnt = df.groupby('user_id', as_index=False)['rating'].agg({"pos_cnt": 'sum'})
    tot_cnt = df.groupby('user_id', as_index=False)['rating'].agg({"tot_cnt": 'count'})
    df = pd.merge(df, pos_cnt, on=['user_id'], how='left')
    df = pd.merge(df, tot_cnt, on=['user_id'], how='left')
    df = df[(df.pos_cnt > 0) & (df.tot_cnt > df.pos_cnt)]
    df = df.drop(['pos_cnt', 'tot_cnt'], axis=1)
    return df
def getDataLoader_sub(data_path, batch_size=32):
    # load train data
    data_fields = ['user_id', 'item_id', 'rating']
    # all data file
    data_df = pd.read_table(data_path, names=data_fields)
    if SMALL:
        data_df = data_df.sample(n=int(len(data_df) * 0.1), replace=False)
    if IMPLICT:
        data_df.rating = (data_df.rating >= 5).astype(np.float32)
    # ua_base = allData.sample(n=90570, replace=False)
    df_train = data_df.sample(n=int(len(data_df) * 0.9), replace=False)
    df_test = data_df.drop(df_train.index, axis=0)
    df_train=drop_df(df_train)
    df_test = drop_df(df_test)
    n_users = max(set(data_df['user_id'].values))+1
    # get item number
    n_items = max(set(data_df['item_id'].values))+1

    print("Initialize end.The user number is:%d,item number is:%d" % (n_users, n_items))
    train_loader = torch.utils.data.DataLoader(
        Interactions(df_train), batch_size=batch_size, shuffle=True)

    test_loader = torch.utils.data.DataLoader(
        Interactions(df_test), batch_size=batch_size, shuffle=False)

    loaders = {'train': train_loader,
               'valid': test_loader}

    return (n_users,n_items ), loaders

In [2]:
input_size, loader=getDataLoader_sub("hin/UIUI.txt", batch_size=2048)


Initialize end.The user number is:943,item number is:1682


In [3]:
model = LFM(input_size[0],input_size[1],device=torch.device("cuda"))
model.fit(loader,5)

(train: 1 ):  13%|######5                                            | 70/543 [00:59<06:46,  1.16it/s, train_loss=6.24]

KeyboardInterrupt: 

In [25]:
n_users*n_items

1586126

In [18]:
data.shape

(1584961,)

In [ ]:
input_size, loader=getDataLoader("../data/ml-100k/u.data")
model = LFM(input_size[0],input_size[1])
model.fit(loader,10)